# Assigment 4 - Geodesic distance fields and paths
## Edoardo Vassallo - S4965918

In [107]:
import igl
import numpy as np
import meshplot as mp

from collections import deque

## 1 - Geodesic Graph

In [108]:
# AUXILLARY FUNCTIONS

# Compute 2D intersection between two segments p1p2 and w1w2
# if not found, return the closest vertex w to p1p2
def intersection_or_closest(p1, p2, w1, w2):

    # AUXILLARY FUNCTION
    # compute distance between a point pt and segment p1p2
    def point_to_seg_dist(pt):
        v = p2 - p1
        t = np.dot(pt - p1, v) / np.dot(v, v)
        t = np.clip(t, 0.0, 1.0)
        proj = p1 + t * v
        return np.linalg.norm(pt - proj)

    # Build direction vectors
    r = p2 - p1
    s = w2 - w1
    q = w1 - p1
    rxs = np.cross(r, s)

    # # If cross ~ 0 → parallel/colinear → no unique intersection
    # rxs = np.cross(r, s)
    # if abs(rxs) < 1e-8:
    #     # return closest of the ws to p1p2
    #     return w1 if point_to_seg_dist(w1) < point_to_seg_dist(w2) else w2

    # Solve for t, u where p1 + t·r = w1 + u·s = x
    t = np.cross(q, s) / rxs
    u = np.cross(q, r) / rxs

    # Check segment bounds
    if (0 <= t <= 1) and (0 <= u <= 1):
        return tuple(p1 + t * r)

    # Otherwise, pick whichever endpoint of w-segment is closest to p‐segment
    return w1 if point_to_seg_dist(w1) < point_to_seg_dist(w2) else w2


# Compute 3d intersection between w1w2 and dual edge p1p2
def compute_intersection(V, v1, v2, w1, w2):

    # Step 1: Define the 2D coordinate system
    w1 = np.asarray(V[w1])
    w2 = np.asarray(V[w2])
    v1 = np.asarray(V[v1])
    v2 = np.asarray(V[v2])

    # w1 and w2 in 2d, we center the space in w1
    w1_2d = np.array([0.0, 0.0])
    w2_2d = np.array([np.linalg.norm(w2 - w1), 0.0])

    # Build orthonormal basis for the plane of 1w1w2
    # x-axis
    x_axis = (w2 - w1)
    x_axis /= np.linalg.norm(x_axis)

    # z-axis (the normal)
    normal = np.cross(x_axis, v1 - w1)
    normal /= np.linalg.norm(normal)

    # y-axis
    y_axis = np.cross(normal, x_axis)

    # Step 2: Project v1 to 2D
    v1_rel = v1 - w1
    v1_2d = np.array([np.dot(v1_rel, x_axis), np.dot(v1_rel, y_axis)])

    # Step 3: Compute 2D coordinates of v2 via intersection of two circles
    r1 = np.linalg.norm(v2 - w1)
    r2 = np.linalg.norm(v2 - w2)
    d = np.linalg.norm(w2_2d - w1_2d)

    # Using circle intersection formula
    # a: offset along the edge direction
    a = (r1**2 - r2**2 + d**2) / (2 * d)
    # h: offset in the perpendicular one
    h = np.sqrt(np.maximum(r1**2 - a**2, 0.0))

    # Midpoint between the circles along w1w2
    p2 = w1_2d + a * (w2_2d - w1_2d) / d

    # Two possible intersection points
    offset = h * np.array([0, 1])
    v2_2d_a = p2 + offset
    v2_2d_b = p2 - offset

    # side where is v1
    side = np.sign(np.cross(w2_2d - w1_2d, v1_2d - w1_2d))
    # Select the intersection on the opposite side to v1
    if np.sign(np.cross(w2_2d - w1_2d, v2_2d_a - w1_2d)) == -side:
        v2_2d = v2_2d_a
    else:
        v2_2d = v2_2d_b
    
    # Step 4: Compute 2D intersection
    x_2d = intersection_or_closest(v1_2d, v2_2d, w1_2d, w2_2d)

    # Step 5: Convert 2D point back to 3D
    x_3d = w1 + x_2d[0] * x_axis + x_2d[1] * y_axis

    return x_3d


In [109]:
# Build graph of geodesic distances on a mesh
def build_geodesic_graph(V, F):
    n = V.shape[0]  # number of vertices
    adj = [[] for _ in range(n)]  # adjacency list: for each vertex, list of (neighbor, weight)

    # Compute triangle-triangle adjacency for dual edges
    TT, _ = igl.triangle_triangle_adjacency(F)

    # For each face
    for f_idx in range(F.shape[0]):
        tri = F[f_idx] 

        # For each corner of the face
        for loc in range(3):
            v  = tri[loc]            # current vertex index
            w1 = tri[(loc + 1) % 3]  # next vertex index around the triangle
            w2 = tri[(loc + 2) % 3]  # other vertex index

            # Primal edges
            for w in (w1, w2):
                d = np.linalg.norm(V[v] - V[w])
                # add primal edge
                adj[v].append((w, d, None)) # (neighbor, weight, dual edge intersection)
                adj[w].append((v, d, None))

            # Dual edges
            f2 = int(TT[f_idx, loc])  # adjacent face across edge opposite v
            if f2 < 0:
                # boundary edge: no adjacent face
                continue

            # find the vertex in face f2 that is not w1 or w2
            edge_vs = {w1, w2}
            vp = next(int(x) for x in F[f2] if int(x) not in edge_vs)

            # Compute vectors for angle calculations at shared edge vertex w1
            v_w1_v  = V[v]  - V[w1]    # vector w1->v
            v_w1_w2 = V[w2] - V[w1]    # vector w1->w2 (edge direction)
            v_w1_vp = V[vp] - V[w1]    # vector w1->vp 

            # norms of these vectors
            n_w1_v  = np.linalg.norm(v_w1_v)
            n_w1_w2 = np.linalg.norm(v_w1_w2)
            n_w1_vp = np.linalg.norm(v_w1_vp)

            # corner angles at w1 in both triangles
            alfa = np.arccos(np.clip(np.dot(v_w1_v, v_w1_w2) / (n_w1_v * n_w1_w2), -1, 1))
            beta = np.arccos(np.clip(np.dot(v_w1_vp, v_w1_w2) / (n_w1_vp * n_w1_w2), -1, 1))

            # Geodesic length between v and vp via w1
            dual_len = np.sqrt(n_w1_v**2 + n_w1_vp**2 - 2 * n_w1_v * n_w1_vp * np.cos(alfa + beta))

            # compute intersection point of the dual edge with the primal edge w1-w2
            intersection = compute_intersection(V, v, vp, w1, w2)
            # add dual edge
            adj[v].append((vp, dual_len, intersection))  # (neighbor, weight, intersection point)
            adj[vp].append((v, dual_len, intersection))  
    return adj

## 2 - Geodesic Solver

In [110]:
# Compute geodesic distances from sources
def geodesic_distances(adj, sources):
    n = len(adj)
    prev = [None] * n  # to store the previous vertex in the path
    dist = np.full(n, np.inf)  # initialize all distances to infinity
    Q = deque()                # deque for SLF-LLL

    # Initialize queue with all source vertices at distance 0
    for s in sources:
        dist[s] = 0.0
        Q.append(s)

    # Propagate distances
    while Q:
        u = Q.popleft()
        du = dist[u]
        # relax all outgoing edges from u
        for v, w, intersection in adj[u]:
            alt = du + w
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = (u, intersection)
                # SLF–LLL: push to front if smaller than current front
                if Q and alt < dist[Q[0]]:
                    Q.appendleft(v)
                else:
                    Q.append(v)

    return dist, prev

In [111]:
# Load the test mesh
V, F = igl.read_triangle_mesh("./data/bunny_1k.obj")
#mp.plot(V, F, shading={"wireframe": True})

In [112]:
# compute geodesic distances
adj = build_geodesic_graph(V, F)

src = [0]

dist, prev = geodesic_distances(adj, src)

# apply color map
plotter = mp.plot(V, F, c=dist, shading={"wireframe": False})

# mark the source with a red point
plotter.add_points(V[[src], :], c=np.array([[1.0, 0.0, 0.0]]), shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001504…

1

## 3. Shortest path between two points

In [113]:
# extract path from target to a source
def extract_geodesic_path(target, sources, prev, V):
    path = []
    current = target
    while current is not None:
        path.append(V[current])
        if current in sources:
            break  # stop if we reach a source
        current, intersection = prev[current]  # get previous vertex and intersection point
        if intersection is not None:
            # dual edge intersection point
            path.append(intersection)    
    return path

def plot_geodesic_path(V, F, dist, path, src, target):
    p = mp.plot(V, F, c=dist, shading={"wireframe": False})

    # mark sources
    p.add_points(V[src, :], c=np.array([[1.0, 0.0, 0.0]]), shading={"point_size": 0.02})
    # mark target
    p.add_points(V[[target], :], c=np.array([[0.0, 1.0, 0.0]]), shading={"point_size": 0.02})

    # draw path
    if len(path) >= 2:
        for i in range(len(path) - 1):
            a = path[i]
            b = path[i+1]
            p.add_lines(a, b, shading={"line_width": 0.05})

    return p


In [120]:
src = [0]
tgt = 300

# Extraxt geodesic path from target to source
path = extract_geodesic_path(tgt, src, prev, V)

# Visualize the geodesic path
plot_geodesic_path(V, F, dist, path, src, tgt)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001504…